# Analyze the perirhinal searchlight cluster

In [1]:
import os
import numpy as np
import pingouin as pg

from tesser import rsa
from tesser import tasks

bids_dir = os.environ['TESSER_BIDS']
res_dir = os.environ['TESSER_RESULTS']
fig_dir = os.path.join(os.environ['TESSER_FIGURES'], 'plots')
os.makedirs(fig_dir, exist_ok=True)
beta_dir = os.path.join(res_dir, 'beta', 'smooth4mm')

/Users/morton/anaconda3/envs/tesser/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


## Load cluster pattern stats

In [2]:
roi = 'gray'
contrast = 'community_group'
stat = 'group'
cluster = 'prcl'
dilate = 2

subjects = tasks.get_subj_list()
patterns = []
for subject in subjects:
    mat, events = rsa.load_cluster_patterns(
        beta_dir, subject, roi, contrast, stat, cluster, dilate
    )
    patterns.append(mat)
sim = rsa.beta_sim_stats(events, patterns)
diff = sim['within'] - sim['across']

## Load induction task performance

In [3]:
induct_raw = tasks.load_induct(bids_dir)
induct = tasks.score_induct(induct_raw)
bias_trial = (
    induct.groupby(['subject', 'trial_type'])['correct']
    .mean()
    .unstack('trial_type')
)
m = induct.groupby('subject')['correct'].mean()
c = bias_trial['central'].to_numpy()
b1 = bias_trial['boundary1'].to_numpy()
b2 = bias_trial['boundary2'].to_numpy()

## Test relationships between pattern similarity and inference

In [4]:
x = np.vstack([np.ones(c.shape), c, b1, b2]).T
y = diff
pg.linear_regression(x, y)

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,-0.007933,0.003440,-2.305856,0.027542,0.170154,0.094714,-0.014933,-0.000934
1,x2,0.009400,0.005868,1.602031,0.118679,0.170154,0.094714,-0.002538,0.021338
2,x3,-0.005839,0.004055,-1.440076,0.159266,0.170154,0.094714,-0.014089,0.002410
3,x4,0.006424,0.003392,1.893874,0.067036,0.170154,0.094714,-0.000477,0.013325


In [5]:
%load_ext watermark
%watermark -i -m -v -iv

Python implementation: CPython
Python version       : 3.7.6
IPython version      : 7.11.1

Compiler    : Clang 4.0.1 (tags/RELEASE_401/final)
OS          : Darwin
Release     : 20.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 12
Architecture: 64bit

pingouin: 0.4.0
tesser  : 0.1.0
numpy   : 1.18.1

